<a href="https://colab.research.google.com/github/Material-Educativo/Tecnicas-heuristicas/blob/main/Recocido_simulado_vs_Alpine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [ ]:
def alpine(x, y):
    """Calcula el valor de la funcion Alpine en 2D."""
    return np.abs(x * np.sin(x) + 0.1 * x) + np.abs(y * np.sin(y) + 0.1 * y)

# Recocido simulado
Veamos el comportamiento de recocido simulado con la función alpine.

In [ ]:
def recocido_simulado(temp_inicial, temp_final, factor_enfriamiento,
                      iteraciones_por_temp):
    """
    Algoritmo de recocido simulado para minimizar la funcion Alpine.
    """
    # Generar solucion inicial aleatoria en [-5, 5] x [-5, 5]
    x_actual = random.uniform(-5, 5)
    y_actual = random.uniform(-5, 5)
    costo_actual = alpine(x_actual, y_actual)

    # Inicializar mejor solucion encontrada
    mejor_x = x_actual
    mejor_y = y_actual
    mejor_costo = costo_actual

    # Almacenar historial para visualizacion posterior
    historial_x = [x_actual]
    historial_y = [y_actual]
    historial_costos = [costo_actual]

    # Establecer temperatura inicial
    temperatura = temp_inicial

    # Bucle externo: proceso de enfriamiento
    while temperatura > temp_final:

        # Estadisticas para esta temperatura (opcional)
        aceptaciones = 0
        mejoras = 0

        # Bucle interno: iteraciones a temperatura constante
        for _ in range(iteraciones_por_temp):

            # Generar solucion vecina con perturbacion pequena
            dx = random.uniform(-0.5, 0.5)
            dy = random.uniform(-0.5, 0.5)
            x_vecino = x_actual + dx
            y_vecino = y_actual + dy

            # Aplicar limites con reflexion en [-5, 5]
            if x_vecino < -5.0:
                x_vecino = -5.0 - (x_vecino + 5.0)
            elif x_vecino > 5.0:
                x_vecino = 5.0 - (x_vecino - 5.0)

            if y_vecino < -5.0:
                y_vecino = -5.0 - (y_vecino + 5.0)
            elif y_vecino > 5.0:
                y_vecino = 5.0 - (y_vecino - 5.0)

            # Evaluar costo de la solucion vecina
            costo_vecino = alpine(x_vecino, y_vecino)

            # Calcular diferencia de costo (delta)
            delta_costo = costo_vecino - costo_actual

            # Criterio de aceptacion de Metropolis
            if delta_costo <= 0:
                # Mejora: aceptar siempre
                aceptar = True
                mejoras += 1
            else:
                # Empeoramiento: aceptar con probabilidad de Boltzmann
                probabilidad = math.exp(-delta_costo / temperatura)
                aceptar = random.random() < probabilidad

            # Aplicar decision de aceptacion
            if aceptar:
                x_actual = x_vecino
                y_actual = y_vecino
                costo_actual = costo_vecino
                aceptaciones += 1

                # Actualizar mejor solucion si es necesario
                if costo_actual < mejor_costo:
                    mejor_x = x_actual
                    mejor_y = y_actual
                    mejor_costo = costo_actual
        # Registrar estado al final de este nivel de temperatura
        # (solo guardamos cuando cambia la temperatura para no saturar memoria)
        historial_x.append(x_actual)
        historial_y.append(y_actual)
        historial_costos.append(costo_actual)

        # Esquema de enfriamiento geometrico
        temperatura = temperatura * factor_enfriamiento

    # Devolver mejor solucion encontrada e historial
    return mejor_x, mejor_y, mejor_costo, historial_x, historial_y, historial_costos

In [ ]:
# === Ejemplo de uso ===

# Configuracion de parametros
temp_inicial = 10.0        # Alta para permitir exploracion amplia
temp_final = 0.01          # Baja para convergencia fina
factor_enfriamiento = 0.90  # Enfriamiento moderado
iteraciones_por_temp = 1000  # Suficiente para equilibrio termico

# Ejecutar algoritmo
mejor_x, mejor_y, mejor_costo, hist_x, hist_y, hist_costos = recocido_simulado(
    temp_inicial,
    temp_final,
    factor_enfriamiento,
    iteraciones_por_temp
)

# Mostrar resultados
print("="*60)
print("RESULTADOS DE RECOCIDO SIMULADO")
print("="*60)
print(f"\nConfiguracion de parametros:")
print(f"  Temperatura inicial:      {temp_inicial}")
print(f"  Temperatura final:        {temp_final}")
print(f"  Factor de enfriamiento:   {factor_enfriamiento}")
print(f"  Iteraciones por temp:     {iteraciones_por_temp}")

print(f"\nMejor solucion encontrada:")
print(f"  x = {mejor_x:.6f}")
print(f"  y = {mejor_y:.6f}")
print(f"  f(x,y) = {mejor_costo:.6f}")

print(f"\nDistancia al optimo global (0,0):")
distancia = np.sqrt(mejor_x**2 + mejor_y**2)
print(f"  {distancia:.6f}")

print(f"\nEstadisticas de ejecucion:")
print(f"  Numero de niveles de temperatura: {len(hist_x)}")
print(f"  Iteraciones totales: {len(hist_x) * iteraciones_por_temp}")

# Veamos la evolución del costo

In [ ]:
# Crear grafica de convergencia
plt.figure(figsize=(10, 6))
iteraciones = range(len(hist_costos))

# Graficar evolucion del costo
plt.plot(iteraciones, hist_costos, 'b-', linewidth=2,
         label='Costo de la solucion actual')
plt.scatter(iteraciones, hist_costos, c='red', s=30, zorder=5)

# Configuracion del grafico
plt.xlabel('Nivel de temperatura (iteracion del bucle externo)', fontsize=12)
plt.ylabel('Costo f(x,y)', fontsize=12)
plt.title('Convergencia de recocido simulado en funcion Alpine',
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

# Veamos su trayectoria

In [ ]:
# === Crear malla para curvas de nivel ===
x_malla = np.linspace(-5.0, 5.0, 200)
y_malla = np.linspace(-5.0, 5.0, 200)
X, Y = np.meshgrid(x_malla, y_malla)
Z = alpine(X, Y)

# === Configurar figura ===
plt.figure(figsize=(12, 10))

# === Graficar curvas de nivel ===
niveles = np.linspace(0, 15, 30)
plt.contour(X, Y, Z, levels=20, colors='gray', alpha=0.5)
contour = plt.contourf(X, Y, Z, levels=niveles, cmap='viridis', alpha=0.6)
plt.colorbar(contour, label='Valor de f(x,y)')

# === Graficar trayectoria del algoritmo ===
plt.plot(
    hist_x,
    hist_y,
    color='red',
    marker='o',
    markersize=6,
    linestyle='-',
    linewidth=2,
    label='Trayectoria del algoritmo',
    alpha=0.8
)

# === Agregar flechas para mostrar dirección ===
for i in range(1, len(hist_x) - 1):
    dx = (hist_x[i] - hist_x[i - 1]) * 0.5
    dy = (hist_y[i] - hist_y[i - 1]) * 0.5
    plt.arrow(
        hist_x[i - 1],
        hist_y[i - 1],
        dx, dy,
        color='black',
        head_width=0.1,
        head_length=0.1,
        zorder=10
    )

# === Marcar puntos importantes ===
plt.plot(
    hist_x[0], hist_y[0],
    'go', markersize=15,
    label='Inicio',
    markeredgecolor='black', markeredgewidth=2
)

# === Configuración de la gráfica ===
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Trayectoria de recocido simulado sobre la función Alpine',
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='upper right')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()


#Veamos una animación

In [ ]:
import matplotlib.animation as animation
from IPython.display import Video  # Para visualizar en Google Colab

In [ ]:
# Crear malla para curvas de nivel
x_vals = np.linspace(-5.0, 5.0, 200)
y_vals = np.linspace(-5.0, 5.0, 200)
X, Y = np.meshgrid(x_vals, y_vals)
Z = alpine(X, Y)

# Configurar figura y ejes
fig, ax = plt.subplots(figsize=(10, 8))

# Graficar curvas de nivel como fondo estatico
niveles = np.linspace(0, 15, 30)
contour = ax.contourf(X, Y, Z, levels=niveles, cmap='viridis', alpha=0.6)
ax.contour(X, Y, Z, levels=20, colors='black', linewidths=0.5)

# Agregar barra de color
plt.colorbar(contour, ax=ax, label='Valor de f(x,y)')

# Marcar el optimo global
ax.plot(0, 0, 'g*', markersize=20, label='Optimo global',
        markeredgecolor='white', markeredgewidth=2, zorder=10)

# Configurar limites y etiquetas
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Proceso de recocido simulado sobre la funcion Alpine',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper right')
ax.grid(True, alpha=0.3)

# Inicializar circulo para la posicion actual
circulo_actual = plt.Circle(
    (hist_x[0], hist_y[0]),
    0.15,  # Radio del circulo
    color='red',
    zorder=15,
    label='Solucion actual'
)
ax.add_artist(circulo_actual)

# Inicializar linea para la trayectoria
linea_trayectoria, = ax.plot(
    [], [],
    'r-',
    linewidth=2,
    alpha=0.6,
    label='Trayectoria'
)

# Actualizar leyenda
ax.legend(fontsize=11, loc='upper right')

# Texto dinamico para mostrar informacion del estado actual
texto_info = ax.text(
    0.02, 0.98,
    '',
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8)
)

def actualizar_frame(frame):
    """
    Actualiza los elementos animados para el frame dado.

    Parameters:
        frame: Indice del frame actual (nivel de temperatura)
    """
    # Actualizar posicion del circulo
    nueva_posicion = (hist_x[frame], hist_y[frame])
    circulo_actual.set_center(nueva_posicion)

    # Actualizar trayectoria (desde inicio hasta frame actual)
    linea_trayectoria.set_data(hist_x[:frame+1], hist_y[:frame+1])

    # Actualizar texto informativo
    temperatura_actual = temp_inicial * (factor_enfriamiento ** frame)
    costo_actual = hist_costos[frame]

    info = f'Iteracion: {frame}\n'
    info += f'Temperatura: {temperatura_actual:.4f}\n'
    info += f'Costo: {costo_actual:.6f}'
    texto_info.set_text(info)

    return circulo_actual, linea_trayectoria, texto_info

# Crear animacion
animacion = animation.FuncAnimation(
    fig,
    actualizar_frame,
    frames=len(hist_x),
    interval=100,  # 100 milisegundos entre frames
    blit=True,
    repeat=True
)

# Guardar como archivo MP4
print("Generando animacion... (esto puede tardar unos momentos)")
animacion.save(
    'recocido_simulado_animacion.mp4',
    writer='ffmpeg',
    fps=10,  # 10 frames por segundo
    dpi=100
)

print("Animacion guardada como: recocido_simulado_animacion.mp4")

# Visualizar en Google Colab
Video('recocido_simulado_animacion.mp4', embed=True, width=800)